In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

In [3]:
def end_to_end_data_prep():
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import os
    from glob import glob
    from PIL import Image
    %matplotlib inline
    base = '/cxldata/skin_disease_1/'
    metadata = pd.read_csv(os.path.join(base,'HAM10000_metadata_orig.csv'))
    #metadata.info()
    #if we combine data into one directory then no need to mention *
    image_path = {os.path.splitext(os.path.basename(x))[0]: x
                  for x in glob(os.path.join('/cxldata/skin_disease_1/HAM10000_images_draftv1/orig/*.jpg'))}
    metadata['path'] = metadata['image_id'].map(image_path.get)
    #upload data into dataset with resize 254,254
    #metadata['image'] = metadata['path'].map(lambda x: np.asarray(Image.open(x).resize((224,224))))
    
    return metadata

In [5]:
metadata = end_to_end_data_prep()


In [6]:
metadata.to_csv('/cxldata/skin_disease_1/sudeep_metadata.csv')

In [2]:
import pandas as pd
df = pd.read_csv('/cxldata/skin_disease_1/sudeep_metadata.csv')

In [5]:
len(df)

10015

In [3]:
df['path'][0]

'/cxldata/skin_disease_1/HAM10000_images_draftv1/orig/ISIC_0027419.jpg'

In [7]:
df.shape[0]

10015

In [4]:
train_image = []
for i in tqdm(range(df.shape[0])):
    img = image.load_img(df['path'][i],target_size=[224,224,3])
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)

X = np.array(train_image)


100%|██████████| 10015/10015 [02:08<00:00, 77.78it/s]


In [5]:
from sklearn.preprocessing import OneHotEncoder
y = df['dx'].values
onehotencoder = OneHotEncoder(categories='auto',sparse=False)
y = onehotencoder.fit_transform(y.reshape(-1,1))

In [9]:
y

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [10]:
X.shape

(10015, 224, 224, 3)

In [11]:
y.shape

(10015, 7)

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=7,test_size=0.2)

In [7]:
X_train.shape

(8012, 224, 224, 3)

In [ ]:
import tensorflow as tf
from tensorflow import keras
base_model = keras.applications.vgg16.VGG16(weights="imagenet",
                                                  include_top=False,input_shape = (224,224,3))
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [7]:
for layer in base_model.layers:
    layer.trainable = False

In [1]:
#optimizer = keras.optimizers.SGD(lr=0.2)
model.compile(loss="categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(X_train,y_train,epochs=3) #changed from 5

NameError: name 'model' is not defined